# Word Embeddings mit `Word2Vec`

Bisher hast du Machine Learning mit Dokumenten durchgeführt. Ähnlichkeiten von Wörtern oder Mehrwort-Kombinationen haben dabei keine Rolle gespielt.

Jetzt versuchst du, eine Darstellung von Wörtern zu finden, mit denen du auch Ähnlichkeiten zwischen ihnen berechnen kannst.

## Daten einladen

Wie gewohnt lädst du die linguistisch analysierten Daten ein:

In [ ]:
!pip install "gensim>=4.0.0"

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

## Embeddings trainieren

Word Embeddings berechnest du am besten mit `gensim`. Wiew gewohnt musst du dazu die Dokumente in ein doppelt geschachteltes Array transformieren.

In [ ]:
import regex as re
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
gensim_words = [[w for w in re.split(r'[\\|\\#]', doc.lower()) if w not in stop_words]
                    for doc in df["nav"]]

Anschließend kannst du das Embedding ausrechnen. Hier ist es so eingestellt, dass nur Wörter berücksichtigt werden, die mindestens fünfmal vorkommen: 

In [ ]:
from gensim.models import Word2Vec
w2v = Word2Vec(gensim_words, min_count=5)

Um die Wortvektoren auch später wieder verwenden zu können, speicherst du sie ab:

In [ ]:
w2v.wv.save_word2vec_format("heise-articles-2020.w2v")

## Ähnlichkeiten abfragen

Nun kannst du mit einfachen Ähnlichkeitsabfragen starten. Wenn dich interessiert, was ähnlich zu `java` ist, kannst du folgende Abfrage ausprobieren:

In [ ]:
w2v.wv.most_similar(positive=["java"], topn=10)

Das Ergebnis ist absolut vernünftig und enthät sowohl konzeptionelle Ähnlichenkeiten (andere Sprachen wie Python und Javascript), aber auch damit verwandte Konzepte (wie Programmiersprache und Entwicklerteam).

Probier es nochmal mit `microsoft` aus:

In [ ]:
w2v.wv.most_similar(positive=["microsoft"], topn=10)

Hoffentlich ist `Corona` kein Thema mehr, wenn du das liest. Im Moment leider schon:

In [ ]:
w2v.wv.most_similar(positive=["corona"], topn=10)

`Word2Vec` kann wie alle Embeddings auch mit Addition und Subtraktion der Vektoren arbeiten und daraus Analogieschlüsse ziehen.

Betrachte dazu folgende Gleichung:

Android - Google = ? - Apple

Durch Umformung kannst du nach `?` auflösen:

? =  Apple + Android - Google

Welche Ergebnis liefert `Word2Vec`?

In [ ]:
pd.DataFrame(w2v.wv.most_similar(positive=["apple", "android"], 
                                   negative=["google"],  topn=10))

Erwartet hätte man `ios`, aber auch die anderen Ergebniss sind schon sehr nah an der richtigen Lösung, weil mit Android durchaus auch Geräte gemeint sein können.

Wenn dir entfallen sein sollte, wie Microsofts Betriebssystem heißt, kannst du `Word2Vec` fragen:

In [ ]:
pd.DataFrame(w2v.wv.most_similar(positive=["microsoft", "android"], 
                                   negative=["google"],  topn=10))

Das Ergebnis ist absolut richtig - und das wurde alles ohne Überwachung trainiert!

## Phrasen

`gensim` kann in dem Korpus auf Phrasen identifizieren und diese statt den Tokens als Vokabular für das Training verwenden:

In [ ]:
from gensim.models import Phrases
entity_transformer = Phrases(gensim_words)
w2vp = Word2Vec(entity_transformer[gensim_words], min_count=5)

Die Ergebnisse für `corona` unterscheiden sich nicht sehr:

In [ ]:
w2vp.wv.most_similar(positive=["corona"], topn=10)

Bei `java` kannst du nun einige Phrasen erkennen, die durch `_` markiert sind. es ist schwer zu entscheiden, ob das Ergebnis wirklich besser ist.

In [ ]:
w2vp.wv.most_similar(positive=["java"], topn=10)

## *Word2Vec* als häufigste Anwendung von Word Embeddings

Häufig wird *Word2Vec* nahezu synonym mit Word Embeddings verwendet - zugegebenermaßen war es auch das erste Word Embedding, was es zu einer signifikanten Verbreitung gebracht hat. Die Performance ist dabei ziemlich gut, sowohl die Berechnung als auch die Abfrage kannst du in (Nahezu-) Echtzeit durchführen (die Abfrage ist bei allen Embeddings gleich schnell).

Anwendungsfallabhängig ist es daher eine gute Idee, wenn du zunächst mit Word2Vec startest und das als Baseline benutzt. Wenn dir die anderen Embeddings, die du später kennenlernen wirst, keine Vorteile bringen, bleibst du einfach bei Word2Vec.